### Random Forest Classifier (in progress)

(Features) **X-variables**: region, date, units sold, production data, climate data, import data, export data, median income.

(Target) **y-variable:** avocado price (numerical outcome)

Note: need more information on y-variable as a price (float).

In [65]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [66]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [67]:
# Import sklearn dependencies
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [68]:
# Load the avocado_mock_df.csv dataset.
avocado_mock_df = pd.read_csv(Path("./MOCK_datasets/MockDataset_Avocado_ML.csv"))
avocado_mock_df.head(10)

,REGION (city),LATITUDE,LONGITUDE,DATE (Month + Year),4046 Units sold,4225 Units,4770 Units,Bags sold,CLIMATE_CA_pcp,CLIMATE_CA_tmin,...,PERU PRODUCTION,COLUMBIA PRODUCTION,IMPORTS BY VALUE ($1000),IMPORTS BY VOLUME (1000 LBS),IMPORTS UNIT VALUE ($ PER POUND),EXPORTS BY VALUE ($1000),EXPORTS BY VOLUME (1000 LBS),EXPORTS UNIT VALUE ($ PER POUND),MEDIAN INCOME,PRICE
0,Los Angeles,34.0522,118.2437,1/1/2017,1223299.39,829896.69,56808.74,1441332.35,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,64251,0.84
1,San Diego,32.7157,117.1611,1/1/2017,149666.20,206181.12,15014.45,189448.58,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,74855,0.93
2,San Francisco,37.7749,122.4194,1/1/2017,260100.73,717625.67,2490.66,66958.27,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,104552,0.95
3,Sacramento,38.5816,121.4944,1/1/2017,144876.31,339664.16,660.97,41564.20,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,63902,0.98
4,Los Angeles,34.0522,118.2437,2/1/2017,5470227.08,1741607.02,937331.61,2701609.82,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,64251,0.53
5,San Diego,32.7157,117.1611,2/1/2017,260418.91,290223.93,20313.95,321784.08,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,74855,0.63
6,San Francisco,37.7749,122.4194,2/1/2017,560686.25,900172.65,21331.13,75785.02,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,104552,0.84
7,Sacramento,38.5816,121.4944,2/1/2017,321474.22,418158.93,11962.44,57213.28,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,63902,0.98
8,Los Angeles,34.0522,118.2437,3/1/2017,2768930.88,897311.08,518369.51,1269943.15,2.19,37.2,...,0,0,224948272,171411.0822,1.312332,4549077,2623.655244,1.733870,64251,0.99
9,San Diego,32.7157,117.1611,3/1/2017,163424.69,176304.67,19186.14,158785.76,2.19,37.2,...,0,0,224948272,171411.0822,1.312332,4549077,2623.655244,1.733870,74855,1.04


In [69]:
# Create our features
dfd = avocado_mock_df.drop(columns=["PRICE"])
X = pd.get_dummies(dfd)

# Create our target
y = avocado_mock_df["PRICE"]

In [70]:
X.describe()

,LATITUDE,LONGITUDE,4046 Units sold,4225 Units,4770 Units,Bags sold,CLIMATE_CA_pcp,CLIMATE_CA_tmin,CLIMATE_CA_tmax,TOTAL_VOLUME,...,EXPORTS BY VOLUME (1000 LBS),EXPORTS UNIT VALUE ($ PER POUND),MEDIAN INCOME,REGION (city)_Los Angeles,REGION (city)_Sacramento,REGION (city)_San Diego,REGION (city)_San Francisco,DATE (Month + Year)_1/1/2017,DATE (Month + Year)_2/1/2017,DATE (Month + Year)_3/1/2017
count,12.000000,12.000000,1.200000e+01,1.200000e+01,12.000000,1.200000e+01,12.000000,12.000000,12.000000,1.200000e+01,...,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,35.781100,119.829650,9.766806e+05,5.887047e+05,135081.766667,5.388443e+05,4.106667,37.100000,56.900000,4.789020e+07,...,1672.898510,1.639240,76890.000000,0.250000,0.250000,0.250000,0.250000,0.333333,0.333333,0.333333
std,2.569249,2.283215,1.605239e+06,4.571479e+05,291290.031332,8.364064e+05,1.860094,0.390803,3.126427,2.148873e+06,...,723.272929,0.096847,17302.863142,0.452267,0.452267,0.452267,0.452267,0.492366,0.492366,0.492366
min,32.715700,117.161100,1.448763e+05,1.763047e+05,660.970000,4.156420e+04,2.190000,36.600000,53.700000,4.567016e+07,...,994.194413,1.513309,63902.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.718075,117.973050,1.631493e+05,2.707826e+05,10282.260000,6.452202e+04,2.190000,36.600000,53.700000,4.567016e+07,...,994.194413,1.513309,64163.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,35.913550,119.869050,2.602598e+05,3.789115e+05,17100.295000,1.245839e+05,3.650000,37.200000,56.100000,4.737148e+07,...,1400.845874,1.670540,69553.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.976575,121.725650,7.263395e+05,8.467503e+05,30200.532500,5.588238e+05,6.480000,37.500000,60.900000,5.062898e+07,...,2623.655244,1.733870,82279.250000,0.250000,0.250000,0.250000,0.250000,1.000000,1.000000,1.000000
max,38.581600,122.419400,5.470227e+06,1.741607e+06,937331.610000,2.701610e+06,6.480000,37.500000,60.900000,5.062898e+07,...,2623.655244,1.733870,104552.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
# Check the balance of our target values
y.value_counts()

0.98    2
0.84    2
1.04    1
0.99    1
0.53    1
1.32    1
0.95    1
0.93    1
0.63    1
1.59    1
Name: PRICE, dtype: int64

In [72]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_test)

Counter({0.95: 1, 0.98: 1, 0.53: 1})

In [73]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)  

ValueError: Unknown label type: 'continuous'

In [74]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  

AttributeError: 'BalancedRandomForestClassifier' object has no attribute 'estimators_'

In [75]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

AttributeError: 'BalancedRandomForestClassifier' object has no attribute 'estimators_'

In [76]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

ValueError: Unknown label type: (2    0.95
3    0.98
4    0.53
Name: PRICE, dtype: float64, array([1.1778, 1.2263, 0.8205]))

In [77]:
# List the features sorted in descending order by feature importance
feature_importance = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
for i in feature_importance:
    print('{} : ({})'.format(i[1], i[0]))

AttributeError: 'BalancedRandomForestClassifier' object has no attribute 'estimators_'